# 🏦 Banco Insights 2.0 - Comprehensive EDA

**Objective**: Explore BACEN data structure to understand:
- Column names, data types, and categorical variables
- Sample data from each dataset (first 10 rows)
- Foreign keys, identifiers, and unique constraints
- Categorical values and business meanings
- Data quality and completeness

**Output**: Complete data dictionary for Banco Insights 2.0

## 📋 Setup and Imports

In [1]:
import pandas as pd
import numpy as np
import json
import warnings
from pathlib import Path
import os
from datetime import datetime

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

# Set up paths
DATA_PATH = Path('../bacen_project_v1/data')
RAW_DATA_PATH = DATA_PATH / 'data_raw_reports'

print("🚀 Banco Insights 2.0 EDA - Setup Complete!")
print(f"📁 Data path: {DATA_PATH.absolute()}")
print(f"📊 Raw reports: {len(list(RAW_DATA_PATH.glob('*.csv')))} quarterly files found")

🚀 Banco Insights 2.0 EDA - Setup Complete!
📁 Data path: /Users/iagoaffonso/code/IagoAffonso/banco-insights-2.0/EDA/../bacen_project_v1/data
📊 Raw reports: 47 quarterly files found


## 📊 Part 1: Data Discovery - File Structure Analysis

In [2]:
# Discover all data files
print("📁 DATA FILES DISCOVERY")
print("=" * 50)

# List all CSV files in data directory
csv_files = list(DATA_PATH.glob('*.csv'))
json_files = list(DATA_PATH.glob('*.json'))
raw_files = list(RAW_DATA_PATH.glob('*.csv'))

print(f"\n📋 Main Data Directory ({len(csv_files)} CSV files):")
for file in csv_files:
    try:
        # Get file size safely
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"  • {file.name} ({size_mb:.1f} MB)")
    except Exception as e:
        print(f"  • {file.name} (size unknown)")

print(f"\n📋 JSON Files ({len(json_files)} files):")
for file in json_files:
    try:
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"  • {file.name} ({size_mb:.1f} MB)")
    except Exception as e:
        print(f"  • {file.name} (size unknown)")

print(f"\n📋 Raw Reports Directory ({len(raw_files)} files):")
print(f"  • Raw quarterly files from {raw_files[0].stem.split('_')[1]} to {raw_files[-1].stem.split('_')[1]}")
print(f"  • Pattern: data_YYYYMM_Tipo2_RelatorioT.csv")

# Focus on main consolidated files for analysis
main_files = {
    'consolidated_cleaned': 'consolidated_cleaned.csv',
    'consolidated_reports': 'consolidated_reports.csv',
    'financial_metrics': 'financial_metrics.csv',
    'financial_metrics_processed': 'financial_metrics_processed.csv',
    'market_metrics': 'market_metrics.csv',
    'credit_data': 'credit_data.csv',
    'cred_pf': 'cred_pf.csv',
    'cred_pj': 'cred_pj.csv'
}

print(f"\n🎯 MAIN FILES FOR ANALYSIS:")
for name, filename in main_files.items():
    file_path = DATA_PATH / filename
    if file_path.exists():
        try:
            size_mb = file_path.stat().st_size / (1024 * 1024)
            print(f"  ✅ {name}: {filename} ({size_mb:.1f} MB)")
        except Exception as e:
            print(f"  ✅ {name}: {filename} (size unknown)")
    else:
        print(f"  ❌ {name}: {filename} (NOT FOUND)")

📁 DATA FILES DISCOVERY

📋 Main Data Directory (8 CSV files):
  • financial_metrics_processed.csv (235.0 MB)
  • cred_pf.csv (89.1 MB)
  • credit_data.csv (242.3 MB)
  • consolidated_cleaned.csv (4561.8 MB)
  • market_metrics.csv (346.9 MB)
  • financial_metrics.csv (1110.8 MB)
  • consolidated_reports.csv (3704.5 MB)
  • cred_pj.csv (153.2 MB)

📋 JSON Files (1 files):
  • consolidated_institutions.json (2.7 MB)

📋 Raw Reports Directory (47 files):
  • Raw quarterly files from 201412 to 201703
  • Pattern: data_YYYYMM_Tipo2_RelatorioT.csv

🎯 MAIN FILES FOR ANALYSIS:
  ✅ consolidated_cleaned: consolidated_cleaned.csv (4561.8 MB)
  ✅ consolidated_reports: consolidated_reports.csv (3704.5 MB)
  ✅ financial_metrics: financial_metrics.csv (1110.8 MB)
  ✅ financial_metrics_processed: financial_metrics_processed.csv (235.0 MB)
  ✅ market_metrics: market_metrics.csv (346.9 MB)
  ✅ credit_data: credit_data.csv (242.3 MB)
  ✅ cred_pf: cred_pf.csv (89.1 MB)
  ✅ cred_pj: cred_pj.csv (153.2 MB)


## 📊 Part 2: Smart Data Loading (Sample-Based Analysis)

In [3]:
def safe_load_sample(file_path, sample_size=10000):
    """
    Safely load a sample of data from a CSV file to avoid memory issues
    """
    try:
        # First, read just the header to understand structure
        header_df = pd.read_csv(file_path, nrows=0)
        total_cols = len(header_df.columns)

        # Read a sample of rows
        sample_df = pd.read_csv(file_path, nrows=sample_size, low_memory=False)

        # Get file info
        file_size_mb = file_path.stat().st_size / (1024 * 1024)

        return {
            'data': sample_df,
            'total_columns': total_cols,
            'sample_rows': len(sample_df),
            'file_size_mb': file_size_mb,
            'success': True,
            'error': None
        }
    except Exception as e:
        return {
            'data': None,
            'total_columns': 0,
            'sample_rows': 0,
            'file_size_mb': 0,
            'success': False,
            'error': str(e)
        }

# Load samples from main files
datasets = {}
print("📊 LOADING DATA SAMPLES (10K rows max per file)")
print("=" * 60)

for name, filename in main_files.items():
    file_path = DATA_PATH / filename
    if file_path.exists():
        result = safe_load_sample(file_path, sample_size=10000)
        if result['success']:
            datasets[name] = result['data']
            print(f"✅ {name}: {result['sample_rows']} rows × {result['total_columns']} cols ({result['file_size_mb']:.1f} MB)")
        else:
            print(f"❌ {name}: Error loading - {result['error']}")
    else:
        print(f"⚠️ {name}: File not found")

print(f"\n📊 Successfully loaded {len(datasets)} datasets for analysis")

📊 LOADING DATA SAMPLES (10K rows max per file)
✅ consolidated_cleaned: 10000 rows × 15 cols (4561.8 MB)
✅ consolidated_reports: 10000 rows × 10 cols (3704.5 MB)
✅ financial_metrics: 10000 rows × 15 cols (1110.8 MB)
✅ financial_metrics_processed: 10000 rows × 10 cols (235.0 MB)
✅ market_metrics: 10000 rows × 15 cols (346.9 MB)
✅ credit_data: 10000 rows × 15 cols (242.3 MB)
✅ cred_pf: 10000 rows × 15 cols (89.1 MB)
✅ cred_pj: 10000 rows × 15 cols (153.2 MB)

📊 Successfully loaded 8 datasets for analysis


## 📊 Part 3: Column Analysis and Data Types

In [4]:
# Comprehensive column analysis
print("🔍 COLUMN ANALYSIS & DATA TYPES")
print("=" * 60)

column_analysis = {}

for name, df in datasets.items():
    print(f"\n📋 {name.upper()}")
    print("-" * 40)
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

    # Column details
    col_info = []
    for col in df.columns:
        dtype = str(df[col].dtype)
        non_null = df[col].count()
        null_count = df[col].isnull().sum()
        null_pct = (null_count / len(df)) * 100
        unique_count = df[col].nunique()

        # Classify column type
        if dtype in ['int64', 'float64', 'int32', 'float32']:
            col_type = 'Numerical'
        elif 'datetime' in dtype:
            col_type = 'DateTime'
        else:
            col_type = 'Categorical/Text'

        col_info.append({
            'column': col,
            'dtype': dtype,
            'type_category': col_type,
            'non_null': non_null,
            'null_count': null_count,
            'null_pct': null_pct,
            'unique_values': unique_count
        })

    column_analysis[name] = col_info

    # Display summary
    col_df = pd.DataFrame(col_info)
    print("\nColumns:")
    for _, row in col_df.iterrows():
        print(f"  • {row['column']:<25} | {row['type_category']:<15} | {row['dtype']:<10} | Unique: {row['unique_values']:<8} | Null: {row['null_pct']:.1f}%")

    # Type summary
    type_summary = col_df['type_category'].value_counts()
    print(f"\nType Summary: {dict(type_summary)}")

🔍 COLUMN ANALYSIS & DATA TYPES

📋 CONSOLIDATED_CLEANED
----------------------------------------
Shape: 10,000 rows × 15 columns

Columns:
  • TipoInstituicao           | Numerical       | int64      | Unique: 1        | Null: 0.0%
  • CodInst                   | Categorical/Text | object     | Unique: 586      | Null: 0.0%
  • AnoMes                    | Categorical/Text | object     | Unique: 1        | Null: 0.0%
  • NomeRelatorio             | Categorical/Text | object     | Unique: 2        | Null: 0.0%
  • NumeroRelatorio           | Numerical       | int64      | Unique: 2        | Null: 0.0%
  • Grupo                     | Categorical/Text | object     | Unique: 3        | Null: 0.0%
  • Conta                     | Numerical       | int64      | Unique: 29       | Null: 0.0%
  • NomeColuna                | Categorical/Text | object     | Unique: 30       | Null: 0.0%
  • DescricaoColuna           | Categorical/Text | object     | Unique: 29       | Null: 0.0%
  • Saldo          

## 📊 Part 4: Sample Data Preview (First 10 Rows)

In [5]:
# Show first 10 rows for each dataset
print("👀 SAMPLE DATA PREVIEW (First 10 rows)")
print("=" * 60)

for name, df in datasets.items():
    print(f"\n📋 {name.upper()} - Sample Data")
    print("-" * 50)

    # Show first 10 rows
    sample_data = df.head(10)
    print(f"Shape: {sample_data.shape}")
    print("\nFirst 10 rows:")
    display(sample_data)

    # Show column list for reference
    print(f"\nColumn Names: {list(df.columns)}")

👀 SAMPLE DATA PREVIEW (First 10 rows)

📋 CONSOLIDATED_CLEANED - Sample Data
--------------------------------------------------
Shape: (10, 15)

First 10 rows:


,TipoInstituicao,CodInst,AnoMes,NomeRelatorio,NumeroRelatorio,Grupo,Conta,NomeColuna,DescricaoColuna,Saldo,AnoMes_M,AnoMes_Q,AnoMes_Y,NomeRelatorio_Grupo_Coluna,NomeInstituicao
0,2,00253448,2014-12-01,Ativo,2,nagroup,78202,Ativo Total Ajustado \n(i) = (a) + (b) + (c) +...,[11000006] + [12000005] + [13000004] + [1600...,1.787970e+08,2014-12,2014Q4,2014,Ativo_nagroup_Ativo Total Ajustado \n(i) = (a)...,BANCO NEON S.A.
1,2,00253448,2014-12-01,Ativo,2,nagroup,78196,Credores por Antecipação de Valor Residual \n(j),[49908008],0.000000e+00,2014-12,2014Q4,2014,Ativo_nagroup_Credores por Antecipação de Valo...,BANCO NEON S.A.
2,2,00253448,2014-12-01,Ativo,2,nagroup,78182,Ativo Total \n(k) = (i) - (j),[10000007]+[20000004],1.787970e+08,2014-12,2014Q4,2014,Ativo_nagroup_Ativo Total \n(k) = (i) - (j),BANCO NEON S.A.
3,2,00259231,2014-12-01,Ativo,2,nagroup,78188,Disponibilidades \n(a),[11000006],7.112886e+05,2014-12,2014Q4,2014,Ativo_nagroup_Disponibilidades \n(a),SICOOB UNIMAIS METROPOLITANA - COOPERATIVA DE ...
4,2,00259231,2014-12-01,Ativo,2,nagroup,78189,Aplicações Interfinanceiras de Liquidez \n(b),[12000005],0.000000e+00,2014-12,2014Q4,2014,Ativo_nagroup_Aplicações Interfinanceiras de L...,SICOOB UNIMAIS METROPOLITANA - COOPERATIVA DE ...
5,2,00259231,2014-12-01,Ativo,2,nagroup,78190,TVM e Instrumentos Financeiros Derivativos \n(c),[13000004],2.362857e+06,2014-12,2014Q4,2014,Ativo_nagroup_TVM e Instrumentos Financeiros D...,SICOOB UNIMAIS METROPOLITANA - COOPERATIVA DE ...
6,2,00259231,2014-12-01,Ativo,2,Operações de Crédito,78191,Operações de Crédito \n(d1),[16000001]-[16900008],1.860031e+08,2014-12,2014Q4,2014,Ativo_Operações de Crédito_Operações de Crédit...,SICOOB UNIMAIS METROPOLITANA - COOPERATIVA DE ...
7,2,00259231,2014-12-01,Ativo,2,Operações de Crédito,78192,Provisão sobre Operações de Crédito \n(d2),[16900008],-7.838668e+06,2014-12,2014Q4,2014,Ativo_Operações de Crédito_Provisão sobre Oper...,SICOOB UNIMAIS METROPOLITANA - COOPERATIVA DE ...
8,2,00259231,2014-12-01,Ativo,2,Operações de Crédito,78193,Operações de Crédito Líquidas de Provisão \n(d),[16000001],1.781644e+08,2014-12,2014Q4,2014,Ativo_Operações de Crédito_Operações de Crédit...,SICOOB UNIMAIS METROPOLITANA - COOPERATIVA DE ...
9,2,00259231,2014-12-01,Ativo,2,Arrendamento Mercantil,78194,Arrendamento Mercantil a Receber \n(e1),[17000000]-[17900007],0.000000e+00,2014-12,2014Q4,2014,Ativo_Arrendamento Mercantil_Arrendamento Merc...,SICOOB UNIMAIS METROPOLITANA - COOPERATIVA DE ...



Column Names: ['TipoInstituicao', 'CodInst', 'AnoMes', 'NomeRelatorio', 'NumeroRelatorio', 'Grupo', 'Conta', 'NomeColuna', 'DescricaoColuna', 'Saldo', 'AnoMes_M', 'AnoMes_Q', 'AnoMes_Y', 'NomeRelatorio_Grupo_Coluna', 'NomeInstituicao']

📋 CONSOLIDATED_REPORTS - Sample Data
--------------------------------------------------
Shape: (10, 10)

First 10 rows:


,TipoInstituicao,CodInst,AnoMes,NomeRelatorio,NumeroRelatorio,Grupo,Conta,NomeColuna,DescricaoColuna,Saldo
0,2,00253448,201412,Ativo,2,NaN,78202,Ativo Total Ajustado \n(i) = (a) + (b) + (c) +...,[11000006] + [12000005] + [13000004] + [1600...,"178797046,86"
1,2,00253448,201412,Ativo,2,NaN,78196,Credores por Antecipação de Valor Residual \n(j),[49908008],0
2,2,00253448,201412,Ativo,2,NaN,78182,Ativo Total \n(k) = (i) - (j),[10000007]+[20000004],"178797046,86"
3,2,00259231,201412,Ativo,2,NaN,78188,Disponibilidades \n(a),[11000006],"711288,56"
4,2,00259231,201412,Ativo,2,NaN,78189,Aplicações Interfinanceiras de Liquidez \n(b),[12000005],0
5,2,00259231,201412,Ativo,2,NaN,78190,TVM e Instrumentos Financeiros Derivativos \n(c),[13000004],"2362856,71"
6,2,00259231,201412,Ativo,2,Operações de Crédito,78191,Operações de Crédito \n(d1),[16000001]-[16900008],"186003113,9"
7,2,00259231,201412,Ativo,2,Operações de Crédito,78192,Provisão sobre Operações de Crédito \n(d2),[16900008],"-7838668,2"
8,2,00259231,201412,Ativo,2,Operações de Crédito,78193,Operações de Crédito Líquidas de Provisão \n(d),[16000001],"178164445,7"
9,2,00259231,201412,Ativo,2,Arrendamento Mercantil,78194,Arrendamento Mercantil a Receber \n(e1),[17000000]-[17900007],0



Column Names: ['TipoInstituicao', 'CodInst', 'AnoMes', 'NomeRelatorio', 'NumeroRelatorio', 'Grupo', 'Conta', 'NomeColuna', 'DescricaoColuna', 'Saldo']

📋 FINANCIAL_METRICS - Sample Data
--------------------------------------------------
Shape: (10, 15)

First 10 rows:


...


Column Names: ['TipoInstituicao', 'CodInst', 'AnoMes', 'NomeRelatorio', 'NumeroRelatorio', 'Grupo', 'Conta', 'NomeColuna', 'DescricaoColuna', 'Saldo', 'AnoMes_M', 'AnoMes_Q', 'AnoMes_Y', 'NomeRelatorio_Grupo_Coluna', 'NomeInstituicao']

📋 CRED_PF - Sample Data
--------------------------------------------------
Shape: (10, 15)

First 10 rows:


,TipoInstituicao,CodInst,AnoMes,NomeRelatorio,NumeroRelatorio,Grupo,Conta,NomeColuna,DescricaoColuna,Saldo,AnoMes_M,AnoMes_Q,AnoMes_Y,NomeRelatorio_Grupo_Coluna,NomeInstituicao
0,2,C0031873,2014-12-01,Carteira de crédito ativa Pessoa Física - moda...,11,nagroup,24452,Total da Carteira de Pessoa Física,Volume de crédito disponibilizado a pessoas fí...,4.879678e+07,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Física - moda...,SOFISA
1,2,C0031873,2014-12-01,Carteira de crédito ativa Pessoa Física - moda...,11,Empréstimo com Consignação em Folha,23227,Total,Total do grupo,2.143650e+06,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Física - moda...,SOFISA
2,2,C0031873,2014-12-01,Carteira de crédito ativa Pessoa Física - moda...,11,Empréstimo sem Consignação em Folha,23235,Total,Total do grupo,2.451427e+07,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Física - moda...,SOFISA
3,2,C0031873,2014-12-01,Carteira de crédito ativa Pessoa Física - moda...,11,Veículos,23251,Total,Total do grupo,3.764477e+06,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Física - moda...,SOFISA
4,2,C0031873,2014-12-01,Carteira de crédito ativa Pessoa Física - moda...,11,Outros Créditos,23275,Total,Total do grupo,1.837438e+07,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Física - moda...,SOFISA
5,2,C0031976,2014-12-01,Carteira de crédito ativa Pessoa Física - moda...,11,nagroup,24452,Total da Carteira de Pessoa Física,Volume de crédito disponibilizado a pessoas fí...,7.101888e+09,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Física - moda...,BRB
6,2,C0031976,2014-12-01,Carteira de crédito ativa Pessoa Física - moda...,11,Empréstimo com Consignação em Folha,23227,Total,Total do grupo,4.115148e+09,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Física - moda...,BRB
7,2,C0031976,2014-12-01,Carteira de crédito ativa Pessoa Física - moda...,11,Empréstimo sem Consignação em Folha,23235,Total,Total do grupo,1.452908e+09,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Física - moda...,BRB
8,2,C0031976,2014-12-01,Carteira de crédito ativa Pessoa Física - moda...,11,Veículos,23251,Total,Total do grupo,1.813139e+08,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Física - moda...,BRB
9,2,C0031976,2014-12-01,Carteira de crédito ativa Pessoa Física - moda...,11,Habitação,23259,Total,Total do grupo,4.451006e+08,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Física - moda...,BRB



Column Names: ['TipoInstituicao', 'CodInst', 'AnoMes', 'NomeRelatorio', 'NumeroRelatorio', 'Grupo', 'Conta', 'NomeColuna', 'DescricaoColuna', 'Saldo', 'AnoMes_M', 'AnoMes_Q', 'AnoMes_Y', 'NomeRelatorio_Grupo_Coluna', 'NomeInstituicao']

📋 CRED_PJ - Sample Data
--------------------------------------------------
Shape: (10, 15)

First 10 rows:


,TipoInstituicao,CodInst,AnoMes,NomeRelatorio,NumeroRelatorio,Grupo,Conta,NomeColuna,DescricaoColuna,Saldo,AnoMes_M,AnoMes_Q,AnoMes_Y,NomeRelatorio_Grupo_Coluna,NomeInstituicao
0,2,C0049944,2014-12-01,Carteira de crédito ativa Pessoa Jurídica - mo...,13,Operações com Recebíveis,23307,Total,Total do grupo,9.944316e+08,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Jurídica - mo...,BTG PACTUAL
1,2,C0049944,2014-12-01,Carteira de crédito ativa Pessoa Jurídica - mo...,13,Comércio Exterior,23339,Total,Total do grupo,7.441903e+08,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Jurídica - mo...,BTG PACTUAL
2,2,C0049944,2014-12-01,Carteira de crédito ativa Pessoa Jurídica - mo...,13,Outros Créditos,23347,Total,Total do grupo,1.100583e+08,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Jurídica - mo...,BTG PACTUAL
3,2,C0049944,2014-12-01,Carteira de crédito ativa Pessoa Jurídica - mo...,13,Financiamento de Infraestrutura/Desenvolviment...,23315,Total,Total do grupo,1.687922e+09,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Jurídica - mo...,BTG PACTUAL
4,2,C0050071,2014-12-01,Carteira de crédito ativa Pessoa Jurídica - mo...,13,nagroup,24453,Total da Carteira de Pessoa Jurídica,Volume de crédito disponibilizado a pessoas ju...,1.634524e+09,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Jurídica - mo...,JOHN DEERE
5,2,C0050071,2014-12-01,Carteira de crédito ativa Pessoa Jurídica - mo...,13,Capital de Giro,23283,Total,Total do grupo,2.575444e+07,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Jurídica - mo...,JOHN DEERE
6,2,C0050071,2014-12-01,Carteira de crédito ativa Pessoa Jurídica - mo...,13,Investimento,23299,Total,Total do grupo,1.106191e+09,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Jurídica - mo...,JOHN DEERE
7,2,C0050071,2014-12-01,Carteira de crédito ativa Pessoa Jurídica - mo...,13,Capital de Giro Rotativo,23291,Total,Total do grupo,1.018445e+05,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Jurídica - mo...,JOHN DEERE
8,2,C0050071,2014-12-01,Carteira de crédito ativa Pessoa Jurídica - mo...,13,Outros Créditos,23347,Total,Total do grupo,1.206247e+08,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Jurídica - mo...,JOHN DEERE
9,2,C0050071,2014-12-01,Carteira de crédito ativa Pessoa Jurídica - mo...,13,Financiamento de Infraestrutura/Desenvolviment...,23315,Total,Total do grupo,1.697310e+07,2014-12,2014Q4,2014,Carteira de crédito ativa Pessoa Jurídica - mo...,JOHN DEERE



Column Names: ['TipoInstituicao', 'CodInst', 'AnoMes', 'NomeRelatorio', 'NumeroRelatorio', 'Grupo', 'Conta', 'NomeColuna', 'DescricaoColuna', 'Saldo', 'AnoMes_M', 'AnoMes_Q', 'AnoMes_Y', 'NomeRelatorio_Grupo_Coluna', 'NomeInstituicao']


## 📊 Part 5: Categorical Variables Analysis

In [6]:
# Analyze categorical variables and their values
print("🏷️ CATEGORICAL VARIABLES ANALYSIS")
print("=" * 60)

categorical_analysis = {}

for name, df in datasets.items():
    print(f"\n📋 {name.upper()}")
    print("-" * 40)

    # Get categorical columns
    categorical_cols = df.select_dtypes(include=['object']).columns
    categorical_data = {}

    for col in categorical_cols:
        unique_values = df[col].value_counts()
        unique_count = len(unique_values)

        print(f"\n🏷️ {col} ({unique_count} unique values):")

        # Show top 10 most frequent values
        top_values = unique_values.head(10)
        for value, count in top_values.items():
            percentage = (count / len(df)) * 100
            print(f"  • {str(value)[:50]:<50} | Count: {count:>6} ({percentage:.1f}%)")

        if unique_count > 10:
            print(f"  ... and {unique_count - 10} more unique values")

        categorical_data[col] = {
            'unique_count': unique_count,
            'top_values': dict(top_values.head(10)),
            'value_counts': dict(unique_values)
        }

    categorical_analysis[name] = categorical_data

    if len(categorical_cols) == 0:
        print("  No categorical columns found in this dataset")

🏷️ CATEGORICAL VARIABLES ANALYSIS

📋 CONSOLIDATED_CLEANED
----------------------------------------

🏷️ CodInst (586 unique values):
  • C0051468                                           | Count:     27 (0.3%)
  • C0051815                                           | Count:     27 (0.3%)
  • C0051482                                           | Count:     26 (0.3%)
  • C0051516                                           | Count:     25 (0.2%)
  • C0051750                                           | Count:     24 (0.2%)
  • C0051781                                           | Count:     24 (0.2%)
  • C0051884                                           | Count:     23 (0.2%)
  • C0051774                                           | Count:     23 (0.2%)
  • 00075847                                           | Count:     23 (0.2%)
  • 00106180                                           | Count:     23 (0.2%)
  ... and 576 more unique values

🏷️ AnoMes (1 unique values):
  • 2014-12-01           

## 📊 Part 6: Identifier and Foreign Key Analysis

In [7]:
# Identify potential keys and identifiers
print("🔑 IDENTIFIER & FOREIGN KEY ANALYSIS")
print("=" * 60)

identifier_analysis = {}

for name, df in datasets.items():
    print(f"\n📋 {name.upper()}")
    print("-" * 40)

    potential_identifiers = []

    for col in df.columns:
        unique_count = df[col].nunique()
        total_count = len(df)
        uniqueness_ratio = unique_count / total_count

        # Classify based on uniqueness and naming patterns
        col_lower = col.lower()

        # Check for identifier patterns
        identifier_type = None
        if uniqueness_ratio == 1.0:
            identifier_type = "Unique Identifier (100% unique)"
        elif uniqueness_ratio > 0.8:
            identifier_type = "High Uniqueness (>80%)"
        elif 'cod' in col_lower or 'id' in col_lower or 'codigo' in col_lower:
            identifier_type = "Code/ID Pattern"
        elif 'nome' in col_lower and uniqueness_ratio > 0.1:
            identifier_type = "Name Field (High Uniqueness)"
        elif 'anomes' in col_lower or 'data' in col_lower or 'ano' in col_lower:
            identifier_type = "Date/Time Identifier"
        elif uniqueness_ratio < 0.1:
            identifier_type = "Low Uniqueness (<10%) - Categorical"

        if identifier_type:
            potential_identifiers.append({
                'column': col,
                'type': identifier_type,
                'unique_count': unique_count,
                'total_count': total_count,
                'uniqueness_ratio': uniqueness_ratio
            })

    identifier_analysis[name] = potential_identifiers

    # Display results
    print("\nPotential Identifiers and Keys:")
    for item in potential_identifiers:
        print(f"  • {item['column']:<25} | {item['type']:<35} | {item['unique_count']:>6}/{item['total_count']:<6} ({item['uniqueness_ratio']:.1%})")

    # Common identifier patterns across datasets
    common_cols = ['CodInst', 'NomeInstituicao', 'AnoMes', 'TipoInstituicao']
    print("\nCommon Identifier Columns:")
    for col in common_cols:
        if col in df.columns:
            unique_count = df[col].nunique()
            print(f"  ✅ {col}: {unique_count} unique values")
        else:
            print(f"  ❌ {col}: Not found")

🔑 IDENTIFIER & FOREIGN KEY ANALYSIS

📋 CONSOLIDATED_CLEANED
----------------------------------------

Potential Identifiers and Keys:
  • TipoInstituicao           | Low Uniqueness (<10%) - Categorical |      1/10000  (0.0%)
  • CodInst                   | Code/ID Pattern                     |    586/10000  (5.9%)
  • AnoMes                    | Date/Time Identifier                |      1/10000  (0.0%)
  • NomeRelatorio             | Low Uniqueness (<10%) - Categorical |      2/10000  (0.0%)
  • NumeroRelatorio           | Low Uniqueness (<10%) - Categorical |      2/10000  (0.0%)
  • Grupo                     | Low Uniqueness (<10%) - Categorical |      3/10000  (0.0%)
  • Conta                     | Low Uniqueness (<10%) - Categorical |     29/10000  (0.3%)
  • NomeColuna                | Low Uniqueness (<10%) - Categorical |     30/10000  (0.3%)
  • DescricaoColuna           | Low Uniqueness (<10%) - Categorical |     29/10000  (0.3%)
  • AnoMes_M                  | Date/Time Ident

## 📊 Part 7: Numerical Variables Summary

In [8]:
# Analyze numerical variables
print("📊 NUMERICAL VARIABLES ANALYSIS")
print("=" * 60)

numerical_analysis = {}

for name, df in datasets.items():
    print(f"\n📋 {name.upper()}")
    print("-" * 40)

    # Get numerical columns
    numerical_cols = df.select_dtypes(include=[np.number]).columns

    if len(numerical_cols) > 0:
        print(f"\nNumerical Columns ({len(numerical_cols)}):")

        numerical_data = {}

        for col in numerical_cols:
            stats = df[col].describe()

            # Additional statistics
            null_count = df[col].isnull().sum()
            null_pct = (null_count / len(df)) * 100
            zero_count = (df[col] == 0).sum()
            negative_count = (df[col] < 0).sum()

            print(f"\n📊 {col}:")
            print(f"  • Count: {stats['count']:,.0f} | Null: {null_count} ({null_pct:.1f}%)")
            print(f"  • Mean: {stats['mean']:,.2f} | Std: {stats['std']:,.2f}")
            print(f"  • Min: {stats['min']:,.2f} | Max: {stats['max']:,.2f}")
            print(f"  • Q1: {stats['25%']:,.2f} | Median: {stats['50%']:,.2f} | Q3: {stats['75%']:,.2f}")
            print(f"  • Zero values: {zero_count} | Negative values: {negative_count}")

            numerical_data[col] = {
                'stats': dict(stats),
                'null_count': null_count,
                'null_pct': null_pct,
                'zero_count': zero_count,
                'negative_count': negative_count
            }

        numerical_analysis[name] = numerical_data
    else:
        print("  No numerical columns found in this dataset")
        numerical_analysis[name] = {}

📊 NUMERICAL VARIABLES ANALYSIS

📋 CONSOLIDATED_CLEANED
----------------------------------------

Numerical Columns (5):

📊 TipoInstituicao:
  • Count: 10,000 | Null: 0 (0.0%)
  • Mean: 2.00 | Std: 0.00
  • Min: 2.00 | Max: 2.00
  • Q1: 2.00 | Median: 2.00 | Q3: 2.00
  • Zero values: 0 | Negative values: 0

📊 NumeroRelatorio:
  • Count: 10,000 | Null: 0 (0.0%)
  • Mean: 2.09 | Std: 0.67
  • Min: 2.00 | Max: 7.00
  • Q1: 2.00 | Median: 2.00 | Q3: 2.00
  • Zero values: 0 | Negative values: 0

📊 Conta:
  • Count: 10,000 | Null: 0 (0.0%)
  • Mean: 77,203.92 | Std: 7,274.75
  • Min: 23,367.00 | Max: 78,202.00
  • Q1: 78,191.00 | Median: 78,195.00 | Q3: 78,198.00
  • Zero values: 0 | Negative values: 0

📊 Saldo:
  • Count: 10,000 | Null: 0 (0.0%)
  • Mean: 577,694,952.95 | Std: 17,633,068,008.59
  • Min: -26,520,346,905.80 | Max: 1,064,674,795,911.33
  • Q1: 0.00 | Median: 551.23 | Q3: 4,908,623.16
  • Zero values: 4452 | Negative values: 517

📊 AnoMes_Y:
  • Count: 10,000 | Null: 0 (0.0%)
  

## 📊 Part 8: Data Quality Assessment

In [9]:
# Comprehensive data quality assessment
print("🔍 DATA QUALITY ASSESSMENT")
print("=" * 60)

quality_summary = {}

for name, df in datasets.items():
    print(f"\n📋 {name.upper()}")
    print("-" * 40)

    # Basic quality metrics
    total_rows = len(df)
    total_columns = len(df.columns)
    total_cells = total_rows * total_columns

    # Missing values
    missing_cells = df.isnull().sum().sum()
    missing_pct = (missing_cells / total_cells) * 100

    # Duplicate rows
    duplicate_rows = df.duplicated().sum()
    duplicate_pct = (duplicate_rows / total_rows) * 100

    # Data type distribution
    numeric_cols = len(df.select_dtypes(include=[np.number]).columns)
    text_cols = len(df.select_dtypes(include=['object']).columns)
    datetime_cols = len(df.select_dtypes(include=['datetime']).columns)

    quality_info = {
        'total_rows': total_rows,
        'total_columns': total_columns,
        'total_cells': total_cells,
        'missing_cells': missing_cells,
        'missing_pct': missing_pct,
        'duplicate_rows': duplicate_rows,
        'duplicate_pct': duplicate_pct,
        'numeric_columns': numeric_cols,
        'text_columns': text_cols,
        'datetime_columns': datetime_cols
    }

    quality_summary[name] = quality_info

    # Display quality metrics
    print(f"📊 Shape: {total_rows:,} rows × {total_columns} columns")
    print(f"📊 Column types: {numeric_cols} numeric, {text_cols} text, {datetime_cols} datetime")
    print(f"📊 Missing values: {missing_cells:,} cells ({missing_pct:.2f}%)")
    print(f"📊 Duplicate rows: {duplicate_rows:,} ({duplicate_pct:.2f}%)")

    # Quality flags
    quality_flags = []
    if missing_pct > 10:
        quality_flags.append(f"⚠️ High missing data: {missing_pct:.1f}%")
    if duplicate_pct > 5:
        quality_flags.append(f"⚠️ High duplicates: {duplicate_pct:.1f}%")
    if missing_pct < 5 and duplicate_pct < 5:
        quality_flags.append("✅ Good data quality")

    for flag in quality_flags:
        print(f"📊 {flag}")

# Overall quality summary
print(f"\n{'='*60}")
print("📊 OVERALL DATA QUALITY SUMMARY")
print(f"{'='*60}")

quality_df = pd.DataFrame(quality_summary).T
print("\nQuality Metrics by Dataset:")
display(quality_df[['total_rows', 'total_columns', 'missing_pct', 'duplicate_pct']].round(2))

🔍 DATA QUALITY ASSESSMENT

📋 CONSOLIDATED_CLEANED
----------------------------------------
📊 Shape: 10,000 rows × 15 columns
📊 Column types: 5 numeric, 10 text, 0 datetime
📊 Missing values: 0 cells (0.00%)
📊 Duplicate rows: 0 (0.00%)
📊 ✅ Good data quality

📋 CONSOLIDATED_REPORTS
----------------------------------------
📊 Shape: 10,000 rows × 10 columns
📊 Column types: 4 numeric, 6 text, 0 datetime
📊 Missing values: 6,172 cells (6.17%)
📊 Duplicate rows: 0 (0.00%)

📋 FINANCIAL_METRICS
----------------------------------------
📊 Shape: 10,000 rows × 15 columns
📊 Column types: 5 numeric, 10 text, 0 datetime
📊 Missing values: 0 cells (0.00%)
📊 Duplicate rows: 0 (0.00%)
📊 ✅ Good data quality

📋 FINANCIAL_METRICS_PROCESSED
----------------------------------------
📊 Shape: 10,000 rows × 10 columns
📊 Column types: 5 numeric, 5 text, 0 datetime
📊 Missing values: 0 cells (0.00%)
📊 Duplicate rows: 0 (0.00%)
📊 ✅ Good data quality

📋 MARKET_METRICS
----------------------------------------
📊 Shape: 10

,total_rows,total_columns,missing_pct,duplicate_pct
consolidated_cleaned,10000.0,15.0,0.00,0.0
consolidated_reports,10000.0,10.0,6.17,0.0
financial_metrics,10000.0,15.0,0.00,0.0
financial_metrics_processed,10000.0,10.0,0.00,0.0
market_metrics,10000.0,15.0,0.00,0.0
credit_data,10000.0,15.0,0.00,0.0
cred_pf,10000.0,15.0,0.00,0.0
cred_pj,10000.0,15.0,0.00,0.0


## 📋 Part 9: Data Dictionary Generation

In [10]:
# Generate comprehensive data dictionary
print("📖 GENERATING DATA DICTIONARY")
print("=" * 60)

data_dictionary = {
    'metadata': {
        'analysis_date': datetime.now().isoformat(),
        'total_datasets': len(datasets),
        'data_source': 'BACEN IFDATA - Brazilian Banking Data'
    },
    'datasets': {}
}

for name, df in datasets.items():
    dataset_info = {
        'description': f'BACEN dataset: {name}',
        'shape': {'rows': len(df), 'columns': len(df.columns)},
        'quality': quality_summary[name],
        'columns': {},
        'sample_data': df.head(3).to_dict('records')
    }

    # Column details
    for col in df.columns:
        col_info = {
            'data_type': str(df[col].dtype),
            'unique_values': int(df[col].nunique()),
            'null_count': int(df[col].isnull().sum()),
            'null_percentage': float((df[col].isnull().sum() / len(df)) * 100)
        }

        # Add type-specific information
        if df[col].dtype in ['int64', 'float64', 'int32', 'float32']:
            col_info['category'] = 'numerical'
            stats = df[col].describe()
            col_info['statistics'] = {
                'min': float(stats['min']) if not pd.isna(stats['min']) else None,
                'max': float(stats['max']) if not pd.isna(stats['max']) else None,
                'mean': float(stats['mean']) if not pd.isna(stats['mean']) else None,
                'std': float(stats['std']) if not pd.isna(stats['std']) else None
            }
        else:
            col_info['category'] = 'categorical'
            # Get top 5 values for categorical columns
            top_values = df[col].value_counts().head(5)
            col_info['top_values'] = dict(top_values.astype(str))

        # Business meaning inference
        col_lower = col.lower()
        if 'cod' in col_lower or 'codigo' in col_lower:
            col_info['business_meaning'] = 'Identifier/Code'
        elif 'nome' in col_lower:
            col_info['business_meaning'] = 'Name/Description'
        elif 'anomes' in col_lower or 'data' in col_lower:
            col_info['business_meaning'] = 'Date/Time Reference'
        elif 'saldo' in col_lower or 'valor' in col_lower:
            col_info['business_meaning'] = 'Financial Amount'
        elif 'tipo' in col_lower:
            col_info['business_meaning'] = 'Classification/Type'
        elif 'relatorio' in col_lower:
            col_info['business_meaning'] = 'Report Information'
        else:
            col_info['business_meaning'] = 'To be determined'

        dataset_info['columns'][col] = col_info

    data_dictionary['datasets'][name] = dataset_info

print("✅ Data dictionary generated successfully!")
print(f"📊 Analyzed {len(datasets)} datasets with {sum(len(d.columns) for d in datasets.values())} total columns")

📖 GENERATING DATA DICTIONARY
✅ Data dictionary generated successfully!
📊 Analyzed 8 datasets with 110 total columns


## 📊 Part 10: Key Findings Summary

In [11]:
# Summarize key findings
print("🎯 KEY FINDINGS SUMMARY")
print("=" * 60)

# Dataset overview
total_records = sum(len(df) for df in datasets.values())
total_columns = sum(len(df.columns) for df in datasets.values())

print(f"\n📊 DATASET OVERVIEW:")
print(f"  • {len(datasets)} main datasets analyzed")
print(f"  • {total_records:,} total records (sample)")
print(f"  • {total_columns} total columns across all datasets")

# Common identifier columns
common_identifiers = ['CodInst', 'NomeInstituicao', 'AnoMes', 'TipoInstituicao']
print(f"\n🔑 COMMON IDENTIFIERS:")
for identifier in common_identifiers:
    datasets_with_id = [name for name, df in datasets.items() if identifier in df.columns]
    if datasets_with_id:
        print(f"  • {identifier}: Found in {len(datasets_with_id)} datasets ({', '.join(datasets_with_id)})")

# Data quality insights
print(f"\n🔍 DATA QUALITY INSIGHTS:")
high_quality_datasets = [name for name, info in quality_summary.items()
                        if info['missing_pct'] < 5 and info['duplicate_pct'] < 5]
print(f"  • {len(high_quality_datasets)} datasets have good quality (low missing/duplicates)")

# Business insights
print(f"\n💼 BUSINESS INSIGHTS:")
financial_columns = []
for name, df in datasets.items():
    for col in df.columns:
        if 'saldo' in col.lower() or 'valor' in col.lower() or 'carteira' in col.lower():
            financial_columns.append(f"{name}.{col}")

print(f"  • {len(financial_columns)} financial amount columns identified")
print(f"  • Time series data available (AnoMes columns)")
print(f"  • Institution-level data perfect for benchmarking")
print(f"  • Multiple report types enable comprehensive analysis")

# Recommendations
print(f"\n🚀 RECOMMENDATIONS FOR BANCO INSIGHTS 2.0:")
recommendations = [
    "Use CodInst as primary institution identifier",
    "Implement time series analysis on AnoMes fields",
    "Create market share calculations using Saldo columns",
    "Build institution profiles using NomeInstituicao",
    "Develop peer comparison features",
    "Implement data quality monitoring",
    "Create automated ETL for raw quarterly reports"
]

for i, rec in enumerate(recommendations, 1):
    print(f"  {i}. {rec}")

print(f"\n{'='*60}")
print("🏆 EDA COMPLETE - Ready for Banco Insights 2.0 Development!")
print(f"{'='*60}")

🎯 KEY FINDINGS SUMMARY

📊 DATASET OVERVIEW:
  • 8 main datasets analyzed
  • 80,000 total records (sample)
  • 110 total columns across all datasets

🔑 COMMON IDENTIFIERS:
  • CodInst: Found in 7 datasets (consolidated_cleaned, consolidated_reports, financial_metrics, market_metrics, credit_data, cred_pf, cred_pj)
  • NomeInstituicao: Found in 7 datasets (consolidated_cleaned, financial_metrics, financial_metrics_processed, market_metrics, credit_data, cred_pf, cred_pj)
  • AnoMes: Found in 8 datasets (consolidated_cleaned, consolidated_reports, financial_metrics, financial_metrics_processed, market_metrics, credit_data, cred_pf, cred_pj)
  • TipoInstituicao: Found in 7 datasets (consolidated_cleaned, consolidated_reports, financial_metrics, market_metrics, credit_data, cred_pf, cred_pj)

🔍 DATA QUALITY INSIGHTS:
  • 7 datasets have good quality (low missing/duplicates)

💼 BUSINESS INSIGHTS:
  • 7 financial amount columns identified
  • Time series data available (AnoMes columns)
  • I

## 📁 Part 11: Export Data Dictionary

In [12]:
# Save data dictionary to JSON file
output_file = 'banco_insights_data_dictionary.json'

# Convert any numpy types to native Python types for JSON serialization
def convert_types(obj):
    if isinstance(obj, dict):
        return {k: convert_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_types(item) for item in obj]
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif pd.isna(obj):
        return None
    else:
        return obj

# Clean data dictionary for JSON export
clean_dictionary = convert_types(data_dictionary)

# Save to file
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(clean_dictionary, f, indent=2, ensure_ascii=False)

print(f"💾 Data dictionary saved to: {output_file}")
print(f"📊 Dictionary contains {len(clean_dictionary['datasets'])} datasets")
print(f"🔍 Total columns documented: {sum(len(ds['columns']) for ds in clean_dictionary['datasets'].values())}")

# Also create a simplified summary CSV
summary_data = []
for dataset_name, dataset_info in clean_dictionary['datasets'].items():
    for col_name, col_info in dataset_info['columns'].items():
        summary_data.append({
            'dataset': dataset_name,
            'column': col_name,
            'data_type': col_info['data_type'],
            'category': col_info['category'],
            'business_meaning': col_info['business_meaning'],
            'unique_values': col_info['unique_values'],
            'null_percentage': round(col_info['null_percentage'], 2)
        })

summary_df = pd.DataFrame(summary_data)
summary_csv = 'banco_insights_columns_summary.csv'
summary_df.to_csv(summary_csv, index=False)

print(f"📋 Column summary saved to: {summary_csv}")
print("\n✅ EDA Complete! Files generated:")
print(f"  • {output_file} - Complete data dictionary")
print(f"  • {summary_csv} - Column summary table")

💾 Data dictionary saved to: banco_insights_data_dictionary.json
📊 Dictionary contains 8 datasets
🔍 Total columns documented: 110
📋 Column summary saved to: banco_insights_columns_summary.csv

✅ EDA Complete! Files generated:
  • banco_insights_data_dictionary.json - Complete data dictionary
  • banco_insights_columns_summary.csv - Column summary table
